In [1]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml

In [2]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [5]:
# add llm api key


# Build a simple directory

In [6]:
simple_glossary = SimpleDirectoryReader(
    r"C:\Users\asarazin\OneDrive - Veltys Max\Documents\documente-marcel\graph"
).load_data()

# Upload Obsidian vault files

#### Using Obsidian reader from LLama Index

In [7]:
#locate obsidian files that make your graph
filepath = r"C:\Users\asarazin\OneDrive - Veltys Max\Documents\documente-marcel\graph"

#load the graph files 
graph_files = ObsidianReader(filepath).load_data()
print(list(graph_files))

[Document(id_='e1abdd8c-6e9c-4bcd-b158-8302eff811b5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='---\nparent: "[[Model]]"\n---\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='a92e2a0d-4ed9-4358-923c-e57df002f598', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='---\nparent: "[[Model]]"\n---\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='47cd7945-39af-4f4b-bd22-b80ce8f15716', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='

# Using manual code to create specific ontology

In [8]:
# Fonction pour extraire les métadonnées YAML et le titre d'un fichier Markdown
def extract_metadata_and_title(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        # Recherchez la section YAML (entre `---` au début et à la fin)
        if content.startswith("---"):
            try:
                yaml_block = content.split("---", 2)[1]
                metadata = yaml.safe_load(yaml_block)
                return os.path.basename(file_path).replace(".md", ""), metadata
            except Exception as e:
                print(f"Erreur lors de l'extraction des métadonnées de {file_path}: {e}")
                return None, None
        return os.path.basename(file_path).replace(".md", ""), None

# Parcourir les fichiers Obsidian et créer des entités et relations
def process_obsidian_notes(directory):
    entities = []
    relations = []
    
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".md"):
                file_path = os.path.join(root, file)
                title, metadata = extract_metadata_and_title(file_path)
                if title:
                    # Ajouter l'entité basée sur le titre
                    entities.append({"name": title})
                    
                    # Ajouter les relations basées sur les propriétés YAML
                    if metadata:
                        for key, value in metadata.items():
                            relations.append({
                                "type": key,  # Nom de la propriété comme relation
                                "source": title,  # Titre du fichier comme entité source
                                "target": value,  # Valeur de la propriété comme entité cible
                            })
    return entities, relations



In [9]:
# Exemple d'utilisation
obsidian_dir = r"C:\Users\asarazin\OneDrive - Veltys Max\Documents\documente-marcel\graph"
entities, relations = process_obsidian_notes(obsidian_dir)

print(relations)

[{'type': 'parent', 'source': 'Classification', 'target': '[[Model]]'}, {'type': 'parent', 'source': 'Clustering', 'target': '[[Model]]'}, {'type': 'parent', 'source': 'Regression', 'target': '[[Model]]'}, {'type': 'type_model', 'source': 'Logistic Regression', 'target': ['[[Classification]]']}, {'type': 'type_model', 'source': 'KNN', 'target': ['[[Clustering]]', '[[Regression]]', '[[Classification]]']}, {'type': 'type_model', 'source': 'Linear Regression', 'target': ['[[Regression]]']}, {'type': 'usage', 'source': 'Linear Regression', 'target': '[[Prediction]]'}, {'type': 'used by', 'source': 'Polynomial regression', 'target': None}, {'type': 'type_model', 'source': 'Polynomial regression', 'target': ['[[Regression]]']}, {'type': 'type_algo', 'source': 'Gradient descent', 'target': '[[Optimization algorithm]]'}]


# Set LLM (OpenAI)

In [10]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings ()

In [11]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Local)

In [12]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [13]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
assistant: As a data governance consultant, I don't have personal preferences or favorites. However, I can tell you about some popular data governance tools that are widely used in the industry. These include:

1. **Collibra**: Known for its comprehensive data governance platform that helps organizations manage data assets, ensure compliance, and improve data quality.

2. **Informatica Axon**: Offers a collaborative data governance solution that integrates with other Informatica products to provide a holistic approach to data management.

3. **Alation**: Provides a data catalog that helps organizations discover, understand, and govern their data assets effectively.

4. **Talend**: Offers data integration and governance solutions that help ensure data quality and compliance across various data sources.

5. **IBM Watson

In [14]:
# Instantiate vector store
simple_glossary = SimpleDirectoryReader(
    r"C:\Users\asarazin\OneDrive - Veltys Max\Documents\documente-marcel\graph"
)

# Instantiate graph store 

In [15]:
graph_store = SimpleGraphStore()
graph_storage_context = StorageContext.from_defaults(graph_store=graph_store)

# Instantiate ontology store

In [16]:
onto_store = SimpleGraphStore()
onto_storage_context = StorageContext.from_defaults(graph_store=onto_store)

# Construct Index

## Construct Vector Store Index

In [17]:
vector_index = VectorStoreIndex.from_documents(
    documents = simple_glossary.load_data(),
    show_progress=True)

Parsing nodes:   0%|          | 0/5 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# save index to disk
vector_index.set_index_id("vector_index")
vector_index.storage_context.persist("vector")

## Construct knowledge graph index 

In [19]:
def process_document(doc):
    # This function will be called for each document
    print(f"Processing document: {doc.doc_id}")
    return doc

In [20]:
# Assuming you have your graph files loaded
print(f"Starting to process {len(graph_files)} documents...")

Starting to process 11 documents...


In [21]:
# Wrap your document processing in a progress bar
graph_index = KnowledgeGraphIndex.from_documents(
        documents=[process_document(doc) for doc in graph_files],  # process one document at a time
        max_triplets_per_chunk=2,
        storage_context=graph_storage_context,
        include_embeddings=True,
    show_progress=True,
)


Processing document: e1abdd8c-6e9c-4bcd-b158-8302eff811b5
Processing document: a92e2a0d-4ed9-4358-923c-e57df002f598
Processing document: 47cd7945-39af-4f4b-bd22-b80ce8f15716
Processing document: 4163a439-34f0-4fb0-b735-b2c3ae18473b
Processing document: 915cbbf2-5f35-463c-b295-bd2f3556d68e
Processing document: c7277ed8-3643-47d5-a34f-4644ebf08724
Processing document: b23f81fd-e34b-429d-bcc5-6b3ec6166a0b
Processing document: 4bb92b5d-341d-4b69-8868-6260a44b50da
Processing document: 8b835cc3-605e-4c99-8232-de4df034db0e
Processing document: 2623ddff-ab84-4b24-a0aa-996d43a9c869
Processing document: 9a74801a-2391-493d-95b7-df92802c38e9


Parsing nodes:   0%|          | 0/11 [00:00<?, ?it/s]

Processing nodes:   0%|          | 0/11 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Generating embeddings: 0it [00:00, ?it/s]

In [22]:
print(graph_index)

In [23]:
graph_index.set_index_id("graph_index")
graph_index.storage_context.persist("knowledge_graph")
graph_storage_context.persist(persist_dir="knowledge_graph")


## Construct knowledge with ontology as assistant

In [24]:
onto_store = SimpleGraphStore()
onto_storage_context = StorageContext.from_defaults(graph_store=onto_store)

In [25]:
# Configurer DynamicLLMPathExtractor avec les entités et relations extraites
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=None,  # Si vous ne limitez pas les types d'entités
    allowed_relation_types=None,  # Si vous ne limitez pas les types de relations
    allowed_entity_props=None,
    allowed_relation_props=None,
)

In [26]:
# Résumé des entités et relations extraites
print("Entités :")
for entity in entities:
    print(entity)

print("\nRelations :")
for relation in relations:
    print(relation)

Entités :
{'name': 'Classification'}
{'name': 'Clustering'}
{'name': 'Model'}
{'name': 'Optimization algorithm'}
{'name': 'Regression'}
{'name': 'Logistic Regression'}
{'name': 'KNN'}
{'name': 'Linear Regression'}
{'name': 'Polynomial regression'}
{'name': 'Gradient descent'}
{'name': 'Prediction'}

Relations :
{'type': 'parent', 'source': 'Classification', 'target': '[[Model]]'}
{'type': 'parent', 'source': 'Clustering', 'target': '[[Model]]'}
{'type': 'parent', 'source': 'Regression', 'target': '[[Model]]'}
{'type': 'type_model', 'source': 'Logistic Regression', 'target': ['[[Classification]]']}
{'type': 'type_model', 'source': 'KNN', 'target': ['[[Clustering]]', '[[Regression]]', '[[Classification]]']}
{'type': 'type_model', 'source': 'Linear Regression', 'target': ['[[Regression]]']}
{'type': 'usage', 'source': 'Linear Regression', 'target': '[[Prediction]]'}
{'type': 'used by', 'source': 'Polynomial regression', 'target': None}
{'type': 'type_model', 'source': 'Polynomial regressi

In [27]:
onto_index = PropertyGraphIndex.from_documents(
    documents=[process_document(doc) for doc in graph_files],
    llm=llm,
    storage_context=onto_storage_context,
    embed_kg_nodes=True,
    kg_extractors=[kg_extractor],
    show_progress=True,
    graph_store=onto_store 

)

Processing document: e1abdd8c-6e9c-4bcd-b158-8302eff811b5
Processing document: a92e2a0d-4ed9-4358-923c-e57df002f598
Processing document: 47cd7945-39af-4f4b-bd22-b80ce8f15716
Processing document: 4163a439-34f0-4fb0-b735-b2c3ae18473b
Processing document: 915cbbf2-5f35-463c-b295-bd2f3556d68e
Processing document: c7277ed8-3643-47d5-a34f-4644ebf08724
Processing document: b23f81fd-e34b-429d-bcc5-6b3ec6166a0b
Processing document: 4bb92b5d-341d-4b69-8868-6260a44b50da
Processing document: 8b835cc3-605e-4c99-8232-de4df034db0e
Processing document: 2623ddff-ab84-4b24-a0aa-996d43a9c869
Processing document: 9a74801a-2391-493d-95b7-df92802c38e9


Parsing nodes:   0%|          | 0/11 [00:00<?, ?it/s]

Extracting and inferring knowledge graph from text:   0%|          | 0/11 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Extracting and inferring knowledge graph from text:   9%|▉         | 1/11 [00:00<00:09,  1.10it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


In [28]:
print(onto_storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x000001F929E3A840>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x000001F929E3A330>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={'type_model': [0.006497118156403303, -0.0020688828080892563, -0.015182276256382465, -0.023121831938624382, 0.047884006053209305, 0.0605301558971405, 0.02881375327706337, 0.021724628284573555, -0.013225730508565903, -0.02258473075926304, 0.017820823937654495, 0.038504309952259064, -0.08663015812635422, 0.07369206100702286, -0.02557627484202385, 0.06643940508365631, 0.03647017851471901, -0.015310654416680336, 0.014782678335905075, 0.019219793379306793, 0.017162563279271126, 0.005282393656671047, -0.004904784262180328, -0.0026288176886737347, -0.0372350737452507, 0.0021551556419581175, 0.0077590192668139935, -0.0010537917

In [29]:
# save index to disk
onto_index.set_index_id("onto_index")
onto_index.storage_context.persist("onto_graph")
onto_storage_context.persist(persist_dir="onto_graph")